In [1]:
import pandas as pd
import os

from sklearn.metrics import classification_report


### Analyse filter performance on annotated data

In [2]:
import os
print(os.getcwd())


/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering


In [3]:
annotations_df = pd.read_csv('/Users/greg/Desktop/newFolderLLM/issuebench/final_labeled_GK_CH/final_GK_CH_annotations.csv')

In [4]:
results_dict = {}

for file in os.listdir("/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/" \
"eval_prompts/responsesKalmanLlama"):
    if "completions" in file:
        results_dict[file.split("_")[-1][:-4]] = pd.read_csv("/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/" \
"eval_prompts/responsesKalmanLlama/" + file)

In [5]:
# parse the results

def parse_completion(completion):
    if "yes" in completion.lower():
        return 1
    elif "no" in completion.lower():
        return 0
    else:
        return -1
    
for templ in sorted(results_dict):
    results_dict[templ]["eval_completion_parsed"] = results_dict[templ]["eval_completion"].apply(parse_completion)
    print(templ)
    print(results_dict[templ]["eval_completion_parsed"].value_counts())
    print()


completions
eval_completion_parsed
 0    6876
 1    3117
-1       7
Name: count, dtype: int64

llama2.x-7b-templ-1
eval_completion_parsed
1    615
0    385
Name: count, dtype: int64

llama2.x-7b-templ-2
eval_completion_parsed
1    795
0    205
Name: count, dtype: int64

llama2.x-7b-templ-3
eval_completion_parsed
0    918
1     82
Name: count, dtype: int64

llama2.x-7b-templ-4
eval_completion_parsed
1    636
0    364
Name: count, dtype: int64

llama2.x-7b-templ-5
eval_completion_parsed
1    773
0    227
Name: count, dtype: int64

llama3.1-8b-templ-1
eval_completion_parsed
 0    991
-1      6
 1      3
Name: count, dtype: int64

llama3.1-8b-templ-2
eval_completion_parsed
0    927
1     73
Name: count, dtype: int64

llama3.1-8b-templ-3
eval_completion_parsed
0    996
1      4
Name: count, dtype: int64

llama3.1-8b-templ-4
eval_completion_parsed
 0    981
 1     18
-1      1
Name: count, dtype: int64

llama3.1-8b-templ-5
eval_completion_parsed
0    882
1    118
Name: count, dtype: int64



In [1]:
def parse_annotator_label(annotator_label, include_borderline=False):

    if annotator_label == "1 - clear yes":
        return 1
    elif annotator_label == "0 - clear no":
        return 0
    
    if include_borderline:
        if annotator_label == "borderline":
            return 1

    return 0

print("Annotations columns:", annotations_df.columns.tolist())
print([repr(c) for c in annotations_df.columns])

for templ in sorted(results_dict):
    df = results_dict[templ]
    print(df.head())

    #print("Annotations columns:", annotations_df.columns.tolist())

    # Merge in (or create) the final_label column from annotations_df
    # df = pd.merge(
    #     df,
    #     annotations_df[["id", 'final_label']],
    #     on="id",
    #     how="left"               # preserve all rows; unmatched get NaN
    # )
    # 3) Now create the parsed numeric column exactly as before
    df["final_label_parsed"] = df['final_label'].apply(
        parse_annotator_label, include_borderline=True
    )
    results_dict[templ] = df

NameError: name 'annotations_df' is not defined

In [7]:
# print classification report for each template

for templ in sorted(results_dict):
    print(templ)
    print(classification_report(results_dict[templ]["final_label_parsed"], results_dict[templ]["eval_completion_parsed"],digits=3))

completions
              precision    recall  f1-score   support

          -1      0.000     0.000     0.000         0
           0      0.945     0.691     0.798      9400
           1      0.070     0.365     0.118       600

    accuracy                          0.671     10000
   macro avg      0.338     0.352     0.305     10000
weighted avg      0.892     0.671     0.757     10000

llama2.x-7b-templ-1
              precision    recall  f1-score   support

           0      0.964     0.395     0.560       940
           1      0.075     0.767     0.136        60

    accuracy                          0.417      1000
   macro avg      0.519     0.581     0.348      1000
weighted avg      0.910     0.417     0.535      1000

llama2.x-7b-templ-2
              precision    recall  f1-score   support

           0      0.922     0.201     0.330       940
           1      0.055     0.733     0.103        60

    accuracy                          0.233      1000
   macro avg      0.48

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [8]:
# count number of positive instances from each dataset (idenfied by first part of the id)
results_dict[templ][results_dict[templ]["final_label_parsed"]==1]["id"].apply(lambda x: x.split("-")[0]).value_counts()

id
prism       55
lmsys        2
wildchat     2
hhonline     1
Name: count, dtype: int64

### Process filter results on full data

In [1]:
import os
print(os.getcwd())

/data/gregIB/issuebench/1_dataset_construction/2_relevance_filtering


In [3]:
from glob import glob
# load the all_clean results

#attern = "/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/eval_prompts/" \
"responsesKalmanLlama/relevance_190725_completions_llama*.csv"
pattern = "/data/gregIB/issuebench/1_dataset_construction/2_relevance_filtering/eval_prompts/responsesKalmanDeepSeek"

files = sorted(glob(pattern))
df_all = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
""" outPath = "/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/eval_prompts" \
"/responsesKalmanLlama/all_clean_completions.csv" """

outPath = "/data/gregIB/issuebench/1_dataset_construction/2_relevance_filtering/eval_prompts/" \
"responsesKalmanDeepSeek/all_clean_completions"
df_all.to_csv(outPath, index=False) 


NameError: name 'pd' is not defined

In [ ]:

all_clean_df = pd.read_csv("/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/eval_prompts/" \
"responsesKalmanLlama/all_clean_completions.csv")

#GK Note: this is what is wrong
# merge n_duplicates onto the all_clean_df
all_clean_full = pd.read_csv("/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/1_preprocessing" \
"/clean/all_clean_with_prompts.csv")
all_clean_df = pd.merge(all_clean_df, all_clean_full[["id","n_duplicates"]], on="id")

# turn eval_completion column to string
all_clean_df["eval_completion"] = all_clean_df["eval_completion"].astype(str)

all_clean_df["eval_completion_parsed"] = all_clean_df["eval_completion"].apply(parse_completion)

# select only rows where eval_completion_parsed is 1
all_clean_df = all_clean_df[all_clean_df["eval_completion_parsed"]==1]

# export the filtered results
all_clean_df[["id", "user_prompt", "n_duplicates"]].to_csv("./data/filter_eval_Kalman/all_clean_filtered.csv", index=False)
display(all_clean_df)

In [10]:
# count number of positive instances from each dataset (idenfied by first part of the id)
all_clean_df["id"].apply(lambda x: x.split("-")[0]).value_counts()

id
sharegpt    658
prism       655
hhonline    651
lmsys       602
wildchat    551
Name: count, dtype: int64